In [5]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

In [6]:
# MINIO
MINIO_ACCESS_KEY = 'hackme'
MINIO_SECRET_KEY = 'becauseiforgottochangethepassword'
MINIO_ENDPOINT = 'http://localhost:9000'

In [8]:
spark = SparkSession.builder.getOrCreate()

23/01/30 02:41:45 WARN Utils: Your hostname, Noels-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.14 instead (on interface en0)
23/01/30 02:41:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/30 02:41:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/30 06:34:51 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 984738 ms exceeds timeout 120000 ms
23/01/30 06:34:51 WARN SparkContext: Killing executors is not supported by current scheduler.


# Create a table

To create a Delta table, write a DataFrame out in the delta format. You can use existing Spark SQL code and change the format from parquet, csv, json, and so on, to delta.

In [ ]:
data = spark.range(0, 5)
data.write.format("delta").save("s3a://delta-lake/demo1")

These operations create a new Delta table using the schema that was inferred from your DataFrame. For the full set of options available when you create a new Delta table, see Create a table and Write to a table.

> Note: This quickstart uses local paths for Delta table locations. For configuring HDFS or cloud storage for Delta tables, see Storage configuration.


# Read data


You read data in your Delta table by specifying the path to the files: "/tmp/delta-table":

In [ ]:
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()

In [ ]:
df.to_pandas_on_spark()

# Update table data

Delta Lake supports several operations to modify tables using standard DataFrame APIs. This example runs a batch job to overwrite the data in the table:

## Overwrite

In [ ]:
data = spark.range(5, 10)
data.write.format("delta").save("/tmp/delta-table", source='parquet', mode='overwrite')

In [ ]:
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()